In [32]:
set.seed(999)
matxMax <- function(mtx) {
  return(arrayInd(which.max(mtx), dim(mtx)))
}
remove_rownames = function(x) {
    for (name in names(x)) rownames(x[[name]]) = NULL
    return(x)
}
handle_nan_etc = function(x) {
  x$bhat[which(is.nan(x$bhat))] = 0
  x$sbhat[which(is.nan(x$sbhat) | is.infinite(x$sbhat))] = 1E3
  return(x)
}
extract_one_data = function(dat, n_random, n_null, infile) {
    if (is.null(dat)) return(NULL)
    na.info = list()
    na.info$n_bhat_ori = nrow(dat$bhat)
    dat$bhat = na.omit(dat$bhat)
    na.info$n_bhat = nrow(dat$bhat)
    na.info$n_sbhat_ori = nrow(dat$sbhat)
    dat$sbhat = na.omit(dat$sbhat)
    na.info$n_sbhat_ori = nrow(dat$sbhat)
    msg = paste(c("Out of ",na.info$n_bhat_ori," SNP, ",na.info$n_bhat," was retained for analysis for ",infile), collapse = "")
    # message(msg)
    z = abs(dat$bhat/dat$sbhat)
    max_idx = matxMax(z)
    if (is.null(max_idx)) stop("No Overlapping SNPs") 
    # strong effect samples
    strong = list(bhat = dat$bhat[max_idx[1],,drop=F], sbhat = dat$sbhat[max_idx[1],,drop=F])
    # random samples excluding the top one
    if (max_idx[1] == 1) {
        sample_idx = 2:nrow(z)
    } else if (max_idx[1] == nrow(z)) {
        sample_idx = 1:(max_idx[1]-1)
    } else {
        sample_idx = c(1:(max_idx[1]-1), (max_idx[1]+1):nrow(z))
    }
    random_idx = sample(sample_idx, min(n_random, length(sample_idx)), replace = F)
    random = list(bhat = dat$bhat[random_idx,,drop=F], sbhat = dat$sbhat[random_idx,,drop=F])
    # null samples defined as |z| < 2
    null.id = which(apply(abs(z), 1, max) < 2)
    if (length(null.id) == 0) {
      warning(paste("Null data is empty for input file", infile))
      null = list()
    } else {
      null_idx = sample(null.id, min(n_null, length(null.id)), replace = F)
      null = list(bhat = dat$bhat[null_idx,,drop=F], sbhat = dat$sbhat[null_idx,,drop=F])
    }
    dat = (list(random = remove_rownames(random), null = remove_rownames(null), strong = remove_rownames(strong)))
    dat$random = handle_nan_etc(dat$random)
    dat$null = handle_nan_etc(dat$null)
    dat$strong = handle_nan_etc(dat$strong)
    return(dat)
}
reformat_data = function(dat, z_only = TRUE) {
    # make output consistent in format with 
    # https://github.com/stephenslab/gtexresults/blob/master/workflows/mashr_flashr_workflow.ipynb      
    res = list(random.z = dat$random$bhat/dat$random$sbhat, 
              strong.z = dat$strong$bhat/dat$strong$sbhat,  
              null.z = dat$null$bhat/dat$null$sbhat)
    if (!z_only) {
      res = c(res, list(random.b = dat$random$bhat,
       strong.b = dat$strong$bhat,
       null.b = dat$null$bhat,
       null.s = dat$null$sbhat,
       random.s = dat$random$sbhat,
       strong.s = dat$strong$sbhat))
  }
  return(res)
}
merge_data = function(res, one_data) {
  if (length(res) == 0) {
      return(one_data)
  } else if (is.null(one_data)) {
      return(res)
  } else {
      for (d in names(one_data)) {
        if (is.null(one_data[[d]])) {
          next
        } else {
            res[[d]] = rbind(res[[d]], one_data[[d]])
        }
      }
      return(res)
  }
}
res = list()
for (f in c('/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257267.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257279.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257298.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257315.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257335.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257337.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257354.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257365.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257379.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257433.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257511.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257524.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257557.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257591.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257594.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257599.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257607.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257621.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257680.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257698.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257704.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257727.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257800.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257815.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257894.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257913.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257923.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257949.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257950.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258010.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258017.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258057.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258162.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258168.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258227.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258232.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258289.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258301.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258302.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258311.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258315.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258376.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258388.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258399.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258405.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258429.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258430.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258441.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258472.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258484.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258498.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258504.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258539.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258559.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258584.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258603.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258610.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258634.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258644.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258653.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258655.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258659.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258674.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258701.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258708.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258727.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258730.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258738.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258768.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258790.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258811.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258818.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258824.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258839.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258890.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258938.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258986.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259024.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259030.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259075.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259153.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259158.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259172.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259175.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259207.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259248.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259251.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259291.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259295.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259319.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259326.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259330.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259357.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259358.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259366.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259385.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259417.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259431.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259448.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259462.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259488.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259494.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259495.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259498.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259523.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259529.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259536.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259577.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259583.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259605.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259623.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259642.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259644.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259660.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259673.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259706.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259709.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259726.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259768.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259775.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259780.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259781.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259802.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259804.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259820.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259823.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259826.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259834.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259863.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259865.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259877.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259881.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259943.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259946.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259948.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259953.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259956.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259959.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259969.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259972.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259985.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259994.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260000.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260005.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260007.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260018.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260032.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260034.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260077.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260083.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260092.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260093.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260097.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260101.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260121.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260163.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260196.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260230.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260231.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260233.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260236.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260238.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260244.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260257.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260260.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260261.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260267.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260274.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260276.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260280.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260285.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260293.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260296.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260306.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260314.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260316.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260317.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260328.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260329.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260337.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260391.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260398.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260400.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260401.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260404.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260412.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260417.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260418.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260428.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260464.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260495.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260526.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260528.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260549.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260565.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260572.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260577.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260578.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260583.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260630.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260634.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260641.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260643.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260645.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260664.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260669.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260686.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260691.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260708.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260729.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260735.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260743.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260772.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260788.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260793.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260804.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260805.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260807.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260816.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260838.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260852.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260855.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260878.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260898.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260903.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260912.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260914.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260916.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260917.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260923.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260924.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260941.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260942.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260948.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260966.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260992.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261003.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261008.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261019.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261026.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261033.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261051.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261052.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261061.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261064.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261067.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261079.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261087.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261094.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261096.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261098.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261115.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261135.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261136.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261147.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261150.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261167.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261168.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261186.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261188.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261195.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261215.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261221.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261236.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261269.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261286.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261292.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261308.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261324.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261325.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261326.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261329.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261338.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261340.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261342.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261351.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261360.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261366.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261371.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261377.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261402.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261408.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261420.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261423.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261439.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261451.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261455.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261460.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261478.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261485.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261490.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261505.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261512.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261520.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261526.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261529.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261534.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261542.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261553.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261556.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261572.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261587.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261594.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261596.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261609.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261611.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261613.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261616.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261617.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261641.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261652.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261654.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261659.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261668.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261670.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261675.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261678.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261701.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261716.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261732.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261737.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261739.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261740.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261770.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261771.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261786.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261796.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261819.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261824.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261829.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261832.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261840.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261879.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261884.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261924.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261934.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261963.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261971.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262096.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262118.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262119.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262155.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262209.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262246.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262312.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262319.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262454.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262468.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262484.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262576.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262580.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262587.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262633.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262655.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262664.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262691.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262700.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262712.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262732.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262833.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262877.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262879.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262881.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262943.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263002.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263072.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263126.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263142.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263155.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263219.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263272.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263276.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263327.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263412.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263465.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263528.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263571.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263603.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263606.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263620.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263715.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263740.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263753.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263809.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263823.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263826.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263843.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263873.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263874.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263934.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263956.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263986.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264112.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264207.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264247.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264364.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264515.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264522.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264538.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264575.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264589.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264668.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264707.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264714.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264772.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264853.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265018.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265043.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265100.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265107.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265148.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265194.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265206.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265218.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265241.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265298.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265354.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265401.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265479.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265491.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265579.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265590.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265666.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265681.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265690.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265713.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265763.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265787.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265808.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265817.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265972.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266028.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266036.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266053.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266066.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266074.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266094.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266173.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266302.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266338.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266371.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266401.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266412.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266469.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266472.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266490.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266524.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266714.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266777.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266820.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266844.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266904.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266910.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266921.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266962.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266964.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266967.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266993.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267002.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267014.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267023.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267040.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267041.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267053.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267058.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267060.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267072.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267080.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267100.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267106.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267107.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267121.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267122.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267125.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267128.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267152.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267160.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267169.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267199.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267216.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267221.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267244.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267248.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267254.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267260.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267272.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267278.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267281.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267296.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267309.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267317.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267321.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267322.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267325.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267365.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267368.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267370.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267385.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267387.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267390.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267454.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267466.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267469.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267470.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267481.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267506.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267508.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267519.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267520.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267529.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267532.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267534.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267575.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267605.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267629.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267632.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267640.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267645.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267651.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267665.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267673.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267680.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267724.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267731.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267740.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267749.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267750.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267787.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267796.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267801.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267838.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267855.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267858.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267868.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267882.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267890.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267904.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267909.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267934.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268006.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268030.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268038.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268043.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268049.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268061.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268119.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268120.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268182.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268189.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268191.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268203.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268205.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268218.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268230.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268362.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268388.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268403.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268460.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268471.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268496.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268516.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268573.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268575.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268592.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268654.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268660.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268713.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268744.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268750.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268751.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268798.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268912.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268996.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269044.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269145.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269176.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269190.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269210.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269243.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269293.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269318.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269352.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269374.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269386.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269388.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269399.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269439.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269473.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269486.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269514.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269609.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269693.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269696.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269699.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269707.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269713.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269737.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269793.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269821.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269825.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269834.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269837.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269858.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269883.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269893.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269896.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269897.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269910.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269929.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269930.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269935.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269937.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269938.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269945.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269956.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269958.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269959.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269968.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269973.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269982.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269983.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269984.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270011.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270020.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270021.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270039.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270049.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270055.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270074.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270091.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270108.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270111.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270112.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270127.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270154.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270157.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270170.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270171.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270175.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270181.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270194.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270231.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270276.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270344.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270362.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270392.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270433.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270504.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270557.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270558.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270562.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270580.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270589.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270607.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270629.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270647.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270681.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270689.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270696.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270765.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270800.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270804.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270820.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270863.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270866.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270882.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270885.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270953.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270959.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271092.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271119.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271122.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271204.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271270.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271303.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271327.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271335.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271344.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271380.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271383.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271425.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271447.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271474.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271538.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271551.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271553.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271576.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271601.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271605.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271614.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271646.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271714.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271723.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271741.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271743.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271755.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271780.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271789.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271795.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271806.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271811.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271816.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271824.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271851.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271855.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271858.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271860.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271862.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271869.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271870.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271880.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271913.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271936.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271971.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271976.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271978.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272008.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272010.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272031.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272037.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272047.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272054.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272068.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272077.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272078.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272079.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272084.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272086.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272092.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272106.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272108.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272121.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272129.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272143.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272145.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272150.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272163.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272168.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272173.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272189.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272195.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272196.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272219.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272235.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272288.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272316.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272320.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272321.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272323.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272325.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272333.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272335.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272338.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272341.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272345.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272356.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272368.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272374.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272375.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272391.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272398.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272405.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272414.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272420.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272425.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272447.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272449.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272455.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272462.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272501.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272505.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272512.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272514.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272556.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272563.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272568.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272573.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272578.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272595.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272599.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272617.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272620.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272630.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272631.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272645.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272654.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272655.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272667.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272674.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272686.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272693.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272695.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272701.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272720.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272734.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272742.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272750.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272752.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272755.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272758.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272760.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272767.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272768.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272777.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272779.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272807.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272808.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272821.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272822.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272831.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272841.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272842.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272853.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272870.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272871.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272884.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272886.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272888.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272894.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272899.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272902.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272909.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272913.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272916.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272933.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272944.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272953.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272973.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272977.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272980.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272983.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272989.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272990.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272994.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273007.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273008.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273015.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273018.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273024.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273032.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273033.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273036.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273038.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273045.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273064.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273066.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273079.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273084.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273108.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273117.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273136.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273142.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273148.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273151.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273156.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273173.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273188.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273192.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273203.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273226.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273230.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273238.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273247.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273253.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273259.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273270.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273271.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273274.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273284.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273287.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273295.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273301.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273311.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273314.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273319.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273329.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273344.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273345.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273367.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273373.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273381.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273382.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273387.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273420.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273432.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273448.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273486.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273487.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273489.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273492.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273540.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273559.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273576.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273599.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273611.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273618.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273619.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273680.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273729.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273749.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273784.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273802.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273841.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273888.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273893.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273899.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274012.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274020.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274021.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274026.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274080.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274104.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274114.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274180.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274184.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274211.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274220.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274253.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274265.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274272.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274286.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274292.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274322.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274349.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274376.uni_weight.rds')) {
  # If cannot read the input for some reason then we just skip it, assuming we have other enough data-sets to use.
  dat = tryCatch(readRDS(f), error = function(e) return(NULL))
  if (is.null(dat)) {
      message(paste("Skip loading file", f, "due to load failure."))
      next
  }
  if (0 > 0 && (ncol(dat$bhat) != 0 || ncol(dat$sbhat) != 0)) {
      message(paste("Skip loading file", f, "because it has", ncol(dat$bhat), "columns different from required", 0))
      next
  }
    res = merge_data(res,reformat_data(tryCatch(extract_one_data(dat, 4, 4, f), error = function(e) return(NULL))))
                                      }

In [5]:
res = merge_data(res, reformat_data(NULL))

In [34]:
a = c('/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257267.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257279.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257298.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257315.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257335.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257337.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257354.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257365.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257379.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257433.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257511.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257524.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257557.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257591.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257594.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257599.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257607.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257621.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257680.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257698.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257704.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257727.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257800.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257815.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257894.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257913.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257923.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257949.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000257950.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258010.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258017.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258057.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258162.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258168.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258227.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258232.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258289.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258301.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258302.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258311.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258315.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258376.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258388.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258399.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258405.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258429.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258430.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258441.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258472.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258484.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258498.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258504.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258539.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258559.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258584.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258603.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258610.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258634.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258644.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258653.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258655.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258659.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258674.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258701.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258708.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258727.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258730.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258738.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258768.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258790.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258811.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258818.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258824.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258839.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258890.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258938.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000258986.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259024.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259030.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259075.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259153.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259158.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259172.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259175.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259207.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259248.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259251.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259291.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259295.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259319.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259326.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259330.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259357.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259358.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259366.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259385.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259417.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259431.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259448.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259462.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259488.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259494.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259495.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259498.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259523.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259529.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259536.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259577.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259583.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259605.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259623.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259642.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259644.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259660.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259673.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259706.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259709.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259726.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259768.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259775.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259780.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259781.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259802.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259804.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259820.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259823.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259826.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259834.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259863.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259865.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259877.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259881.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259943.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259946.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259948.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259953.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259956.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259959.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259969.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259972.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259985.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000259994.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260000.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260005.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260007.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260018.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260032.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260034.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260077.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260083.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260092.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260093.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260097.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260101.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260121.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260163.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260196.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260230.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260231.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260233.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260236.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260238.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260244.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260257.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260260.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260261.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260267.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260274.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260276.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260280.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260285.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260293.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260296.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260306.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260314.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260316.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260317.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260328.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260329.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260337.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260391.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260398.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260400.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260401.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260404.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260412.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260417.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260418.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260428.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260464.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260495.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260526.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260528.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260549.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260565.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260572.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260577.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260578.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260583.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260630.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260634.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260641.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260643.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260645.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260664.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260669.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260686.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260691.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260708.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260729.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260735.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260743.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260772.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260788.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260793.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260804.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260805.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260807.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260816.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260838.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260852.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260855.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260878.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260898.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260903.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260912.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260914.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260916.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260917.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260923.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260924.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260941.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260942.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260948.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260966.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000260992.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261003.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261008.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261019.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261026.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261033.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261051.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261052.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261061.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261064.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261067.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261079.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261087.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261094.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261096.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261098.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261115.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261135.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261136.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261147.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261150.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261167.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261168.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261186.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261188.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261195.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261215.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261221.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261236.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261269.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261286.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261292.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261308.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261324.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261325.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261326.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261329.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261338.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261340.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261342.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261351.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261360.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261366.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261371.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261377.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261402.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261408.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261420.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261423.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261439.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261451.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261455.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261460.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261478.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261485.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261490.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261505.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261512.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261520.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261526.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261529.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261534.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261542.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261553.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261556.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261572.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261587.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261594.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261596.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261609.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261611.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261613.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261616.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261617.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261641.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261652.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261654.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261659.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261668.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261670.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261675.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261678.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261701.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261716.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261732.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261737.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261739.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261740.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261770.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261771.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261786.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261796.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261819.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261824.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261829.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261832.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261840.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261879.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261884.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261924.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261934.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261963.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000261971.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262096.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262118.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262119.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262155.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262209.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262246.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262312.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262319.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262454.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262468.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262484.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262576.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262580.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262587.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262633.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262655.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262664.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262691.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262700.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262712.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262732.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262833.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262877.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262879.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262881.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000262943.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263002.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263072.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263126.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263142.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263155.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263219.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263272.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263276.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263327.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263412.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263465.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263528.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263571.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263603.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263606.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263620.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263715.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263740.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263753.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263809.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263823.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263826.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263843.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263873.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263874.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263934.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263956.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000263986.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264112.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264207.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264247.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264364.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264515.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264522.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264538.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264575.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264589.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264668.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264707.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264714.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264772.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264853.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000264920.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265018.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265043.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265100.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265107.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265148.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265194.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265206.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265218.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265241.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265298.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265354.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265401.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265479.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265491.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265579.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265590.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265666.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265681.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265690.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265713.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265763.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265787.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265808.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265817.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000265972.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266028.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266036.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266053.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266066.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266074.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266094.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266173.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266302.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266338.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266371.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266401.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266412.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266469.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266472.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266490.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266524.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266714.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266777.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266820.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266844.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266904.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266910.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266921.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266962.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266964.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266967.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000266993.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267002.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267014.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267023.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267040.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267041.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267053.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267058.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267060.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267072.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267080.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267100.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267106.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267107.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267121.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267122.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267125.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267128.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267152.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267160.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267169.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267199.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267216.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267221.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267244.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267248.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267254.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267260.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267272.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267278.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267281.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267296.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267309.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267317.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267321.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267322.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267325.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267365.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267368.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267370.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267385.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267387.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267390.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267454.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267466.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267469.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267470.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267481.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267506.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267508.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267519.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267520.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267529.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267532.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267534.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267575.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267605.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267629.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267632.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267640.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267645.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267651.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267665.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267673.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267680.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267724.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267731.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267740.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267749.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267750.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267787.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267796.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267801.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267838.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267855.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267858.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267868.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267882.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267890.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267904.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267909.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267934.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000267940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268006.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268030.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268038.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268043.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268049.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268061.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268119.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268120.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268182.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268189.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268191.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268203.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268205.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268218.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268230.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268362.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268388.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268403.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268460.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268471.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268496.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268516.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268573.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268575.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268592.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268654.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268660.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268713.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268744.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268750.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268751.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268798.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268912.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268947.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000268996.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269001.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269044.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269069.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269145.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269176.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269190.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269210.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269243.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269293.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269318.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269352.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269374.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269386.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269388.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269399.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269439.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269473.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269486.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269514.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269609.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269693.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269696.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269699.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269707.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269713.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269737.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269793.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269821.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269825.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269834.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269837.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269858.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269883.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269893.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269896.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269897.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269910.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269929.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269930.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269935.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269937.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269938.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269940.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269945.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269956.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269958.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269959.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269968.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269973.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269982.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269983.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000269984.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270011.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270020.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270021.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270039.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270049.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270055.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270074.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270091.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270108.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270111.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270112.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270127.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270154.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270157.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270170.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270171.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270175.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270181.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270194.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270231.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270276.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270344.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270362.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270392.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270426.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270433.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270504.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270557.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270558.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270562.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270580.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270589.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270607.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270629.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270647.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270681.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270689.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270696.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270765.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270800.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270804.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270820.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270863.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270866.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270882.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270885.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270953.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000270959.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271092.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271119.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271122.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271204.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271270.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271303.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271327.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271335.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271344.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271380.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271383.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271425.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271447.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271461.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271474.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271538.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271551.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271553.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271576.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271601.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271605.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271614.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271646.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271714.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271723.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271741.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271743.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271755.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271780.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271789.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271795.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271806.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271811.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271816.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271824.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271851.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271855.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271858.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271860.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271862.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271869.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271870.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271880.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271895.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271913.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271936.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271971.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271976.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000271978.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272008.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272010.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272031.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272037.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272047.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272054.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272056.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272068.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272077.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272078.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272079.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272084.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272086.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272092.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272106.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272108.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272121.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272129.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272143.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272145.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272150.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272163.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272168.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272173.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272189.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272195.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272196.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272219.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272235.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272288.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272316.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272320.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272321.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272323.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272325.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272333.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272335.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272338.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272341.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272343.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272345.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272356.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272368.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272374.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272375.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272391.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272398.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272405.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272414.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272420.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272425.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272447.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272449.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272455.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272462.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272501.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272505.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272512.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272514.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272556.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272563.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272568.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272573.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272578.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272595.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272599.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272617.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272620.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272630.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272631.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272645.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272654.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272655.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272667.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272674.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272686.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272693.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272695.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272701.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272720.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272734.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272742.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272750.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272752.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272755.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272758.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272760.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272767.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272768.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272777.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272779.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272807.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272808.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272821.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272822.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272831.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272841.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272842.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272853.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272870.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272871.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272884.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272886.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272888.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272894.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272899.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272902.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272909.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272913.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272916.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272918.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272933.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272944.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272953.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272973.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272977.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272980.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272983.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272989.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272990.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000272994.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273007.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273008.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273015.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273018.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273024.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273032.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273033.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273036.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273038.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273045.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273064.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273066.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273079.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273084.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273108.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273117.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273136.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273142.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273148.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273151.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273156.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273173.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273188.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273192.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273203.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273226.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273230.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273238.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273247.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273253.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273259.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273270.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273271.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273274.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273284.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273287.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273295.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273301.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273311.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273314.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273319.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273329.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273344.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273345.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273367.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273372.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273373.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273381.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273382.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273387.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273419.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273420.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273432.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273448.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273456.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273486.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273487.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273489.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273492.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273540.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273559.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273576.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273599.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273604.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273611.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273618.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273619.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273680.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273702.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273729.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273749.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273784.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273802.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273841.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273888.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273891.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273893.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000273899.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274012.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274020.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274021.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274026.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274070.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274080.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274104.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274114.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274180.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274184.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274211.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274220.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274253.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274265.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274272.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274286.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274292.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274322.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274349.uni_weight.rds','/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274376.uni_weight.rds')

In [37]:
f

[1] "/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/RDS/geneTpmResidualsAgeGenderAdj_rename.ENSG00000274376.uni_weight.rds"

In [38]:
reformat_data(tryCatch(extract_one_data(dat, 4, 4, f), error = function(e) return(NULL)))

AC,DLPFC,PCC,TWAS_sumstat
-1.8936322,-2.932749,-1.0275372,3.0866667
0.6426589,-2.502219,0.6866269,-0.3938053
0.3530550,-1.410273,-2.3155480,-1.6590476
0.5443367,0.607413,0.1106476,3.0243902
AC,DLPFC,PCC,TWAS_sumstat
-1.148272,1.032391,4.319652,0.03571429
AC,DLPFC,PCC,TWAS_sumstat
-0.7478948,1.9542814,-0.5033773,-1.1847134
-0.4603528,1.3545744,0.3428691,0.3326403
0.8623479,-0.2552369,-0.7230758,0.8040541


In [36]:
3977/995

[1] 3.996985

In [13]:
library("tidyverse")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

